# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
! pip install -q schedule pytest

In [7]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [8]:
import re


def get_book_data(book_url):
    """
    Получает данные о книге с указанной страницы каталога Books to Scrape.
    
    Функция парсит HTML-страницу книги и извлекает всю основную информацию:
    название, цену, рейтинг, количество в наличии, описание и дополнительные 
    характеристики из таблицы Product Information.
    
    Parameters
    ----------
    book_url : str
        URL-адрес страницы книги для парсинга
        
    Returns
    -------
    dict
        Словарь с данными о книге, содержащий следующие ключи:
        - 'title': название книги (str)
        - 'price': цена книги (str)
        - 'rating': рейтинг книги (str)
        - 'stock': количество в наличии (str)
        - 'description': описание книги (str)
        - 'upc': универсальный код товара (str)
        - 'product_type': тип товара (str)
        - 'price_excl_tax': цена без налога (str)
        - 'price_incl_tax': цена с налогом (str)
        - 'tax': сумма налога (str)
        - 'availability': информация о наличии (str)
        - 'number_of_reviews': количество отзывов (str)
    """
    try:
        response = requests.get(book_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Название (с проверкой)
        title_element = soup.find('h1')
        title = title_element.text.strip() if title_element else "Нет названия"
        
        # Цена (с проверкой)
        price_element = soup.find('p', class_='price_color')
        price = price_element.text.strip() if price_element else "Нет цены"
        
        # Рейтинг (с проверкой)
        rating_element = soup.find('p', class_='star-rating')
        if rating_element and len(rating_element.get('class', [])) > 1:
            rating = rating_element['class'][1]
        else:
            rating = "No rating"
        
        # В наличии (с проверкой)
        stock_element = soup.find('p', class_='instock availability')
        stock = "0"
        if stock_element:
            stock_text = stock_element.text
            stock_match = re.search(r'\((\d+) available\)', stock_text)
            if stock_match:
                stock = stock_match.group(1)
        
        # Описание (с проверкой)
        description_element = soup.find('div', id='product_description')
        description = "Нет описания"
        if description_element:
            description_sibling = description_element.find_next_sibling('p')
            if description_sibling:
                description = description_sibling.text.strip()
        
        # Таблица с дополнительной информацией (с проверкой)
        product_info = {}
        table = soup.find('table', class_='table table-striped')
        if table:
            rows = table.find_all('tr')
            for row in rows:
                header_element = row.find('th')
                value_element = row.find('td')
                if header_element and value_element:
                    header = header_element.text.strip()
                    value = value_element.text.strip()
                    product_info[header] = value
        
        book_data = {
            'title': title,
            'price': price,
            'rating': rating,
            'stock': stock,
            'description': description,
            'upc': product_info.get('UPC', ''),
            'product_type': product_info.get('Product Type', ''),
            'price_excl_tax': product_info.get('Price (excl. tax)', ''),
            'price_incl_tax': product_info.get('Price (incl. tax)', ''),
            'tax': product_info.get('Tax', ''),
            'availability': product_info.get('Availability', ''),
            'number_of_reviews': product_info.get('Number of reviews', '')
        }
        
        return book_data
    
    except Exception as e:
        print(f"Ошибка в get_book_data: {e}")
        return {
            'title': f'Ошибка: {str(e)[:50]}',
            'price': '0',
            'rating': 'Zero',
            'stock': '0',
            'description': 'Ошибка загрузки',
            'upc': '',
            'product_type': '',
            'price_excl_tax': '',
            'price_incl_tax': '',
            'tax': '',
            'availability': '',
            'number_of_reviews': ''
        }

In [9]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': '£51.77',
 'rating': 'Three',
 'stock': '22',
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put yo

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [10]:
from tqdm import tqdm

def scrape_books(is_save=False):
    """
        Парсит данные обо всех книгах со всех страниц каталога Books to Scrape.
    
    Функция последовательно проходит по всем страницам каталога, извлекает ссылки
    на отдельные книги и собирает о них данные с помощью функции get_book_data.
    
    Parameters
    ----------
    is_save : bool, optional
        Флаг сохранения результатов в файл. Если True, данные сохраняются
        в файл books_data.txt в текущей папке. По умолчанию False.
        
    Returns
    -------
    List[Dict]
        Список словарей, где каждый словарь содержит данные об одной книге.
        Структура словаря соответствует возвращаемому значению get_book_data.
    """
    all_books = []
    book_urls = []
    page_number = 1
    
    # Собираем все URL книг без прогресс-бара
    while True:
        try:
            if page_number == 1:
                url = "http://books.toscrape.com/index.html"
            else:
                url = f"http://books.toscrape.com/catalogue/page-{page_number}.html"

            response = requests.get(url, timeout=10)
            if response.status_code == 404:
                break

            soup = BeautifulSoup(response.text, 'html.parser')
            books = soup.find_all('article', class_='product_pod')

            if not books:
                break

            # Собираем URL книг с текущей страницы
            for book in books:
                link_tag = book.find('h3').find('a')
                if link_tag:
                    link = link_tag['href']
                    if link.startswith('../../../'):
                        full_url = link.replace('../../../', 'http://books.toscrape.com/catalogue/')
                    elif link.startswith('../'):
                        full_url = link.replace('../', 'http://books.toscrape.com/catalogue/')
                    elif link.startswith('catalogue/'):
                        full_url = 'http://books.toscrape.com/' + link
                    else:
                        full_url = 'http://books.toscrape.com/catalogue/' + link
                    book_urls.append(full_url)

            # Проверяем есть ли следующая страница
            next_button = soup.find('li', class_='next')
            if not next_button:
                break

            page_number += 1
            time.sleep(0.1)

        except Exception as e:
            print(f"Ошибка при поиске страниц: {e}")
            break
    
    print(f"Всего найдено книг: {len(book_urls)}")
    

    with tqdm(total=len(book_urls), desc="Обработка книг", unit="книга") as pbar:
        for book_url in book_urls:
            try:
                book_info = get_book_data(book_url)
                all_books.append(book_info)
                pbar.set_postfix(book=book_info['title'][:20])
                time.sleep(0.1)
            except Exception as e:
                pbar.set_postfix(error="Ошибка")
            finally:
                pbar.update(1)
    
    # Сохраняем в файл 
    if is_save and all_books:
        try:
            with open('books_data.txt', 'w', encoding='utf-8') as f:
                for i, (book_url, book) in enumerate(zip(book_urls, all_books), 1):
                    f.write(f"URL: {book_url}\n")
                    f.write(str(book))
                    
            print(f"Сохранено {len(all_books)} книг в файл books_data.txt")
            
        except Exception as e:
            print(f"Ошибка при сохранении: {e}")
    elif is_save:
        print("Нет данных для сохранения")
    
    print(f"Обработано книг: {len(all_books)}")
    return all_books

In [11]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

Всего найдено книг: 1000


Обработка книг: 100%|█████████████████████████████████| 1000/1000 [07:11<00:00,  2.32книга/s, book=1,000 Places to See]

Сохранено 1000 книг в файл books_data.txt
Обработано книг: 1000
<class 'list'> 1000


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [13]:
import schedule
import time
from datetime import datetime

def schedule_scrape_books() -> None:
    """
    Автоматически парсит книги со всех страниц сайта Books to Scrape.
    Расписание:
    - Основной запуск: ежедневно в 19:00
    """
    # Настройка ежедневного автоматического парсинга в 19:00
    schedule.every().day.at('19:00').do(scrape_books, is_save=True)

    # Проверки выполнения запланированных задач (каждые 60 секунд)
    while True:
        schedule.run_pending()
        time.sleep(60)

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [16]:
import pytest
import sys
import os

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from scraper import get_book_data, scrape_books


class TestBookParser:
    """Тесты для функций парсинга книг"""

    def test_get_book_data_returns_dict(self):
        """Тест: get_book_data возвращает словарь"""
        # Используем реальный URL для тестирования
        test_url = "http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
        result = get_book_data(test_url)

        assert isinstance(result, dict), "Функция должна возвращать словарь"

    def test_get_book_data_has_required_keys(self):
        """Тест: словарь содержит все необходимые ключи"""
        test_url = "http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
        result = get_book_data(test_url)

        required_keys = [
            'title', 'price', 'rating', 'stock', 'description',
            'upc', 'product_type', 'price_excl_tax', 'price_incl_tax',
            'tax', 'availability', 'number_of_reviews'
        ]

        for key in required_keys:
            assert key in result, f"Отсутствует ключ: {key}"

    def test_get_book_data_title_not_empty(self):
        """Тест: название книги не пустое"""
        test_url = "http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
        result = get_book_data(test_url)

        assert result['title'] != "", "Название книги не должно быть пустым"
        assert len(result['title']) > 0, "Название книги должно содержать текст"
        assert result['title'] != "Нет названия", "Название должно быть получено"

    def test_scrape_books_returns_list(self):
        """Тест: scrape_books возвращает список"""
        # Ограничиваем сбор 1 страницей для быстрого теста
        result = scrape_books(is_save=False)

        assert isinstance(result, list), "Функция должна возвращать список"

    def test_scrape_books_contains_books(self):
        """Тест: scrape_books возвращает непустой список книг"""
        result = scrape_books(is_save=False)

        assert len(result) > 0, "Список книг не должен быть пустым"
        assert all(isinstance(book, dict) for book in result), "Все элементы должны быть словарями"


if __name__ == "__main__":
    # Запуск тестов вручную
    test_class = TestBookParser()

    print("ЗАПУСК АВТОТЕСТОВ")

    tests = [
        ("get_book_data возвращает словарь", test_class.test_get_book_data_returns_dict),
        ("словарь содержит все необходимые ключи", test_class.test_get_book_data_has_required_keys),
        ("название книги не пустое", test_class.test_get_book_data_title_not_empty),
        ("scrape_books возвращает список", test_class.test_scrape_books_returns_list),
        ("scrape_books возвращает непустой список книг", test_class.test_scrape_books_contains_books),
    ]

    passed = 0
    failed = 0

    for test_name, test_func in tests:
        try:
            test_func()
            print(f"PASS: {test_name}")
            passed += 1
        except Exception as e:
            print(f"FAIL: {test_name}")
            print(f"   Ошибка: {e}")
            failed += 1

    print(f"Результат: {passed} passed, {failed} failed")

    if failed == 0:
        print("Все тесты прошли успешно!")
    else:
        print("Некоторые тесты не прошли")


NameError: name '__file__' is not defined

In [15]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest test/test_scraper.py

============================= test session starts =============================
platform win32 -- Python 3.9.7, pytest-8.4.2, pluggy-1.5.0
rootdir: C:\Users\79688
plugins: anyio-2.2.0
collected 0 items

============================== warnings summary ===============================
D:\Anaconda\lib\site-packages\pyreadline\py3k_compat.py:8
  D:\Anaconda\lib\site-packages\pyreadline\py3k_compat.py:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
    return isinstance(x, collections.Callable)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
============================= 1 warning in 0.09s ==============================


ERROR: file or directory not found: test/test_scraper.py



## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```